In [1]:
!pip install azure-storage-blob # Microoft Azure
!pip install pyarrow
!pip install psycopg2 sqlalchemy

ERROR: Invalid requirement: '#'


     ---------------------------------------- 21.5/21.5 MB 6.2 MB/s eta 0:00:00


     ---------------------------------------- 1.2/1.2 MB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 7.0 MB/s eta 0:00:00
     -------------------------------------- 291.4/291.4 kB 6.0 MB/s eta 0:00:00


In [2]:
!pip install typing-extensions --upgrade

In [3]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

ImportError: cannot import name 'TypeAliasType' from 'typing_extensions' (C:\Users\Andrea\Anaconda3\lib\site-packages\typing_extensions.py)

In [4]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [8]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'violation'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

accumulated_df = pd.DataFrame()
violation_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Accumulate the data
    accumulated_df = pd.concat([accumulated_df, df], ignore_index=True)
    print(accumulated_df.shape)


data_chunk_offset_102000000.csv
(5000000, 19)
data_chunk_offset_107000000.csv
(10000000, 19)
data_chunk_offset_111000000.csv
(13923718, 19)
data_chunk_offset_14000000.csv


MemoryError: Unable to allocate 256. KiB for an array with shape (32768,) and data type int64

In [ ]:
violation_df = accumulated_df.copy()